<a href="https://colab.research.google.com/github/alejandro56664-adl/unal-data-mining/blob/main/Tarea3_JheisonMorales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Minería de Datos - 2021-I
## Tarea 3 - Asociación

![logo_unal](http://seeklogo.com/images/U/Universidad_Nacional_de_Colombia_-_Sede_Bogot_and__225_-logo-A05EAD6D0F-seeklogo.com.png)

PROFESORA ELIZABETH LEON GUZMAN

Presentado por:
- [Jheison Alejandro Morales Vásquez](mailto:jhmoralesva@unal.edu.co)



In [1]:
!pip install pyfpgrowth
!pip install efficient-apriori

     |████████████████████████████████| 1.6MB 6.2MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5477 sha256=808d7d17ae11c71b95b1109139eb0554934604e66bd8c4baf3a8d25a0359a116
  Stored in directory: /root/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth


In [2]:
import numpy as np
import pandas as pd
import time
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from scipy import stats
from google.colab import files

In [3]:
import pyfpgrowth
from efficient_apriori import apriori

## 1.

Por cada un de las siguientes preguntas,

proveer un ejemplo de una regla de asociación del dominio de “market basket” que satisface las siguientes condiciones. También, describir si las reglas son interesantes (subjetivamente).

- Una regla que que tiene alto soporte y alta confianza
- Una regla que tiene razonablemente alto soporte pero baja confianza
- Una regla que tiene bajo soporte y baja confianza (d) Una regla que tiene bajo soporte y alta confianza

## 2.

¿Por qué el proceso de descubrimiento de reglas de asociación es relativamente simple comparado con la generación de grandes conjuntos de ítems en bases de datos transaccionales?

## 3.

Considere el siguiente conjunto de datos:


In [4]:
dic_3 = { 
        'TID': ['T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08', 'T09', 'T10'],
        'Items': ['milk, beer, diapers',
                  'bread, butter, milk',
                  'milk, diapers, cookies',
                  'bread, butter, cookies',
                  'beer, cookies, diapers',
                  'milk, diapers, bread, butter',
                  'bread, butter, diapers',
                  'beer, diapers',
                  'milk, diapers, bread, butter',
                  'beer, cookies']
       }
df_items_3 = pd.DataFrame.from_dict(dic_3)
df_items_3.set_index('TID', inplace=True)
df_items_3.head()

,Items
TID,
T01,"milk, beer, diapers"
T02,"bread, butter, milk"
T03,"milk, diapers, cookies"
T04,"bread, butter, cookies"
T05,"beer, cookies, diapers"


- ¿cuál es el número máximo de reglas de asociación que se pueden generar? (incluyendo reglas con soporte 0)

In [5]:
def get_list_transactions(s):
  result = []
  all = []
  values = s.values
  for i in range(len(values)):
    values_aux = values[i].split(', ')
    result.append(values_aux)
    all = all + values_aux
  
  items = list(set(all))
  
  return result, items


In [6]:
list_transactions, items = get_list_transactions(df_items_3['Items'])


In [7]:
list_transactions

[['milk', 'beer', 'diapers'],
 ['bread', 'butter', 'milk'],
 ['milk', 'diapers', 'cookies'],
 ['bread', 'butter', 'cookies'],
 ['beer', 'cookies', 'diapers'],
 ['milk', 'diapers', 'bread', 'butter'],
 ['bread', 'butter', 'diapers'],
 ['beer', 'diapers'],
 ['milk', 'diapers', 'bread', 'butter'],
 ['beer', 'cookies']]

In [8]:
items

['diapers', 'bread', 'milk', 'butter', 'beer', 'cookies']

In [9]:
d = len(items)
d

6

In [10]:
R = 3**d + 2**(d + 1) + 1
R

858

- ¿Cuál es el tamaño máximo de los conjuntos de items frecuentes que se pueden extraer (asumir el umbral de minsoporte > 0)?


In [11]:
2**d

64

- Escribir una regla que contenga 3 ítems que se genere de este conjunto de datos.

  - R/: { milk, diapers} → { beer }

In [12]:
#TODO optimizar esto con lambdas o closures
def get_all_3_itemsets(items):
  itemset = []
  k = 3
  d = len(items)
  d_k = d - k + 1 #para contar desde 0
  for i in range(0, d_k):
    for j in range(i+1, d_k + 1):
      for k1 in range(j+1, d_k + 2):
        itemset.append([items[i], items[j], items[k1]])
  return itemset, k

def get_all_2_itemsets(items):
  itemset = []
  k = 2
  d = len(items)
  d_k = d - k + 1 #para contar desde 0
  for i in range(0, d_k):
    for j in range(i+1, d_k + 1):
        itemset.append([items[i], items[j]])
  return itemset, k

def get_all_1_itemsets(items):
  itemset = []
  k = 1
  d = len(items)
  d_k = d - k + 1 #para contar desde 0
  for i in range(0, d_k):
        itemset.append([items[i]])
  return itemset, k


- Encontrar un conjunto de items (de tamaño mayor a 2) con el valor de soporte máximo.

In [13]:
k3_itemsets, _ = get_all_3_itemsets(items)
k3_itemsets

[['diapers', 'bread', 'milk'],
 ['diapers', 'bread', 'butter'],
 ['diapers', 'bread', 'beer'],
 ['diapers', 'bread', 'cookies'],
 ['diapers', 'milk', 'butter'],
 ['diapers', 'milk', 'beer'],
 ['diapers', 'milk', 'cookies'],
 ['diapers', 'butter', 'beer'],
 ['diapers', 'butter', 'cookies'],
 ['diapers', 'beer', 'cookies'],
 ['bread', 'milk', 'butter'],
 ['bread', 'milk', 'beer'],
 ['bread', 'milk', 'cookies'],
 ['bread', 'butter', 'beer'],
 ['bread', 'butter', 'cookies'],
 ['bread', 'beer', 'cookies'],
 ['milk', 'butter', 'beer'],
 ['milk', 'butter', 'cookies'],
 ['milk', 'beer', 'cookies'],
 ['butter', 'beer', 'cookies']]

In [14]:
def get_support(k_itemsets, transactions):
  # implemetnación por fuerza bruta
  l = len(k_itemsets)
  supports = np.zeros(l)
  for t in transactions:
    for i in range(l):
      i_s = k_itemsets[i]
      if len(t) > len(i_s):
        supports[i] = supports[i] + int(set(i_s).issubset(t))
  return supports
        

In [15]:
k3_itemsets_supp = get_support(k3_itemsets, list_transactions)
k3_itemsets_supp

array([2., 2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [16]:
df_support_3 = pd.DataFrame({'support_count': k3_itemsets_supp, 'k3_itemsets': k3_itemsets})
df_support_3[ df_support_3['support_count'] > 0]

,support_count,k3_itemsets
0,2.0,"[diapers, bread, milk]"
1,2.0,"[diapers, bread, butter]"
4,2.0,"[diapers, milk, butter]"
10,2.0,"[bread, milk, butter]"


- Encontrar un par de ítems (a y b) tal que las reglas a → b y b → a tengan la misma confianza.

In [17]:
def get_support_df(items, list_transactions, fn_get_k_itemsets):
  k_itemsets, k = fn_get_k_itemsets(items)
  k_itemsets_s = get_support(k_itemsets, list_transactions)
  N = len(list_transactions)
  l = len(k_itemsets)
  df = pd.DataFrame({'k_itemsets': k_itemsets, 'k': np.ones(l)*k, 'support_count': k_itemsets_s})
  df['support'] = df['support_count']/N
  return df

In [18]:
df_support_2 = get_support_df(items, list_transactions, get_all_2_itemsets)

#filtramos los que tienen support_count > 0
df_support_2 = df_support_2[ df_support_2['support_count'] > 0]
df_support_2

,k_itemsets,k,support_count,support
0,"[diapers, bread]",2.0,3.0,0.3
1,"[diapers, milk]",2.0,4.0,0.4
2,"[diapers, butter]",2.0,3.0,0.3
3,"[diapers, beer]",2.0,2.0,0.2
4,"[diapers, cookies]",2.0,2.0,0.2
5,"[bread, milk]",2.0,3.0,0.3
6,"[bread, butter]",2.0,5.0,0.5
8,"[bread, cookies]",2.0,1.0,0.1
9,"[milk, butter]",2.0,3.0,0.3
10,"[milk, beer]",2.0,1.0,0.1


In [19]:
df_support_1 = get_support_df(items, list_transactions, get_all_1_itemsets)

#desempaquetamos del array ['item'] -> 'item'
df_support_1['k_itemsets'] = df_support_1['k_itemsets'].map(lambda s: s[0])

df_support_1

,k_itemsets,k,support_count,support
0,diapers,1.0,7.0,0.7
1,bread,1.0,5.0,0.5
2,milk,1.0,5.0,0.5
3,butter,1.0,5.0,0.5
4,beer,1.0,4.0,0.4
5,cookies,1.0,4.0,0.4


In [20]:
# Ahora que ya vimos que funciona creamos un closure con acceso al diccionario
def add_a_b_metrics(k1_itemset):

  #convertimos a el df a diccionario
  dict_support = dict(zip(k1_itemset['k_itemsets'].values, k1_itemset['support_count'].values))

  print(dict_support)
  #retornamos un closure con acceso al diccionario
  def add_metrics_cols(row):

    #calculamos la confianza = supp(a->b)/supp(a)
    itemset = row['k_itemsets'] # [a, b]
    
    supp_a_b = row['support_count']

    supp_a = dict_support[itemset[0]]
    supp_b = dict_support[itemset[1]]
    

    row['c(a->b)'] = supp_a_b/supp_a
    row['c(b->a)'] = supp_a_b/supp_b
    row['I(a,b)'] = supp_a_b/(supp_b*supp_a)
    return row
  
  return add_metrics_cols

In [21]:
df_support_2_metrics = df_support_2.apply(add_a_b_metrics(df_support_1), axis=1)
df_support_2_metrics

{'diapers': 7.0, 'bread': 5.0, 'milk': 5.0, 'butter': 5.0, 'beer': 4.0, 'cookies': 4.0}


,k_itemsets,k,support_count,support,c(a->b),c(b->a),"I(a,b)"
0,"[diapers, bread]",2.0,3.0,0.3,0.428571,0.60,0.085714
1,"[diapers, milk]",2.0,4.0,0.4,0.571429,0.80,0.114286
2,"[diapers, butter]",2.0,3.0,0.3,0.428571,0.60,0.085714
3,"[diapers, beer]",2.0,2.0,0.2,0.285714,0.50,0.071429
4,"[diapers, cookies]",2.0,2.0,0.2,0.285714,0.50,0.071429
5,"[bread, milk]",2.0,3.0,0.3,0.600000,0.60,0.120000
6,"[bread, butter]",2.0,5.0,0.5,1.000000,1.00,0.200000
8,"[bread, cookies]",2.0,1.0,0.1,0.200000,0.25,0.050000
9,"[milk, butter]",2.0,3.0,0.3,0.600000,0.60,0.120000
10,"[milk, beer]",2.0,1.0,0.1,0.200000,0.25,0.050000


In [22]:
df_support_2_metrics[ df_support_2_metrics['c(a->b)'] == df_support_2_metrics['c(b->a)']]

,k_itemsets,k,support_count,support,c(a->b),c(b->a),"I(a,b)"
5,"[bread, milk]",2.0,3.0,0.3,0.60,0.60,0.1200
6,"[bread, butter]",2.0,5.0,0.5,1.00,1.00,0.2000
9,"[milk, butter]",2.0,3.0,0.3,0.60,0.60,0.1200
14,"[beer, cookies]",2.0,1.0,0.1,0.25,0.25,0.0625


## 4. 

Dado la siguiente base de datos X:


In [23]:
dic_4 = { 
        'TID': ['T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08'],
        'Items': ['A, B, C, D',
                  'A, C, D, F',
                  'C, D, E, G, A',
                  'A, D, F, B',
                  'B, C, G',
                  'D, F, G',
                  'A, B, G',
                  'C, D, F, G']
       }
df_items_4 = pd.DataFrame(dic_4)
df_items_4.set_index('TID', inplace=True)
df_items_4

,Items
TID,
T01,"A, B, C, D"
T02,"A, C, D, F"
T03,"C, D, E, G, A"
T04,"A, D, F, B"
T05,"B, C, G"
T06,"D, F, G"
T07,"A, B, G"
T08,"C, D, F, G"


Usando valores de umbral de soporte = 25 % y confianza = 60 %, encuentre:
- Todos los conjuntos de ítems en la base de datos X


In [24]:
list_transactions_4, items_4 = get_list_transactions(df_items_4['Items'])

In [25]:
list_transactions_4

[['A', 'B', 'C', 'D'],
 ['A', 'C', 'D', 'F'],
 ['C', 'D', 'E', 'G', 'A'],
 ['A', 'D', 'F', 'B'],
 ['B', 'C', 'G'],
 ['D', 'F', 'G'],
 ['A', 'B', 'G'],
 ['C', 'D', 'F', 'G']]

In [26]:
df_p4_support_2 = get_support_df(items_4, list_transactions_4, get_all_2_itemsets)
df_p4_support_3 = get_support_df(items_4, list_transactions_4, get_all_3_itemsets)

df_p4_support = pd.concat([df_p4_support_3, df_p4_support_2])

#filtramos los que tienen support_count > 0
df_p4_support = df_p4_support[ (df_p4_support['support'] > 0.25) & (df_p4_support['k'] == 2)]
df_p4_support

,k_itemsets,k,support_count,support
0,"[A, D]",2.0,4.0,0.500
2,"[A, C]",2.0,3.0,0.375
3,"[A, B]",2.0,3.0,0.375
6,"[D, F]",2.0,4.0,0.500
7,"[D, C]",2.0,4.0,0.500
10,"[D, G]",2.0,3.0,0.375
17,"[C, G]",2.0,3.0,0.375


In [27]:
df_p4_support_1 = get_support_df(items_4, list_transactions_4, get_all_1_itemsets)

#desempaquetamos del array ['item'] -> 'item'
df_p4_support_1['k_itemsets'] = df_p4_support_1['k_itemsets'].map(lambda s: s[0])

df_p4_support_1

,k_itemsets,k,support_count,support
0,A,1.0,5.0,0.625
1,D,1.0,6.0,0.750
2,F,1.0,4.0,0.500
3,C,1.0,5.0,0.625
4,B,1.0,4.0,0.500
5,E,1.0,1.0,0.125
6,G,1.0,5.0,0.625


- Reglas de asociación fuertes para la base de datos X


In [28]:
df_p4_support_metrics = df_p4_support.apply(add_a_b_metrics(df_p4_support_1), axis=1)
df_p4_support_metrics

{'A': 5.0, 'D': 6.0, 'F': 4.0, 'C': 5.0, 'B': 4.0, 'E': 1.0, 'G': 5.0}


,k_itemsets,k,support_count,support,c(a->b),c(b->a),"I(a,b)"
0,"[A, D]",2.0,4.0,0.500,0.800000,0.666667,0.133333
2,"[A, C]",2.0,3.0,0.375,0.600000,0.600000,0.120000
3,"[A, B]",2.0,3.0,0.375,0.600000,0.750000,0.150000
6,"[D, F]",2.0,4.0,0.500,0.666667,1.000000,0.166667
7,"[D, C]",2.0,4.0,0.500,0.666667,0.800000,0.133333
10,"[D, G]",2.0,3.0,0.375,0.500000,0.600000,0.100000
17,"[C, G]",2.0,3.0,0.375,0.600000,0.600000,0.120000


In [29]:
df_p4_support_metrics[(df_p4_support_metrics['c(a->b)'] > 0.6) | (df_p4_support_metrics['c(b->a)'] > 0.6)]

,k_itemsets,k,support_count,support,c(a->b),c(b->a),"I(a,b)"
0,"[A, D]",2.0,4.0,0.500,0.800000,0.666667,0.133333
3,"[A, B]",2.0,3.0,0.375,0.600000,0.750000,0.150000
6,"[D, F]",2.0,4.0,0.500,0.666667,1.000000,0.166667
7,"[D, C]",2.0,4.0,0.500,0.666667,0.800000,0.133333


- Analice las asociaciones engañosas para el conjunto de reglas obtenido en el numeral anterior.

**Notas**: Engañosas son las reglas que tienen un valor interesante de 'confianza' sin embargo pueden ser resultado del azar mas que de un comportamiento real reflejado por los datos. 


Al revisar el lift(I) vemos que las reglas las engañosas son las que estan bajo del umbral 1, por lo que se puede concluir que la relación es debil, aunque aparecen juntos C->D y D->F aparecen aún con menos frecuencia de lo que se esperaría por azar.

## 5. 

El algoritmo Apriori usa estrategías de generación y conteo para derivar conjuntos de items frecuentes. 

| id | items   |
| -- | ------- |
| 1  | a,b,d,e |
| 2  | b,c,d   |
| 3  | a,b,d,e |
| 4  | a,c,d,e |
| 5  | b,c,d,e |
| 6  | b,d,e   |
| 7  | c,d     |
| 8  | a,b,c   |
| 9  | a,d,e   |
| 10 | b,d     |

Conjuntos de item de tamaño k + 1 son creados de conjuntos de items de tamaño k. Un conjunto candidato es eliminado si uno de sus subconjuntos no es frecuente en la fase de poda. Supongamos que el algoritmo Apriori es aplicado a los datos de la siguiente tabla con un soporte mínimo de 30% (ejercicio realizado en clase).

- Dibujar el lattice de los conjuntos de items y etiquetar cada nodo con las siguientes letras:
  - i. N: si el conjunto no es considerado candidato
  - ii. F: si el conjunto candidato es frecuente
  - iii. I: si el conjunto candidato no es frecuente
- ¿Cuál es el porcentaje de conjuntos de items frecuente?
- ¿Cuál es el radio de poda en este conjunto de datos? (El radio de poda es definido como el porcentaje de conjuntos de items no considerados a ser candidatos, ya sea por que no son generados durante la etapa de generación de candidatos, o por que son podados en la etapa de poda).
- ¿Cuál es la rata de falsa alarma? (porcentaje de los conjuntos de items candidatos que son encontrados NO frecuentes después de calcular el soporte).

**Notas**: frecuentes se queda, identificar los no frecuentes,
los q se podan son los de N
F, I, candidatos pero no son frecuentes, al no ser frecuentes se eliminan, en adelante es poda
Frecuentes: pasan el umbral del soporte
al no pasar el umbral no es frecuente. N es el q se poda

In [30]:
dic_5 = { 
        'TID': [i for i in range(10)],
        'Items': ['a, b, d, e',
                  'b, c, d',
                  'a, b, d, e',
                  'a, c, d, e',
                  'b, c, d, e',
                  'b, d, e',
                  'c, d',
                  'a, b, c',
                  'a, d, e',
                  'b, d']
       }
df_items_5 = pd.DataFrame(dic_5)
df_items_5.set_index('TID', inplace=True)
df_items_5

,Items
TID,
0,"a, b, d, e"
1,"b, c, d"
2,"a, b, d, e"
3,"a, c, d, e"
4,"b, c, d, e"
5,"b, d, e"
6,"c, d"
7,"a, b, c"
8,"a, d, e"


## 6. 

Dada la siguiente base de datos transaccional Y:


In [31]:
dic_6 = { 
        'TID': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
        'Items': ['A, B, C',
                  'A, C, D, E',
                  'A, B, D',
                  'A, C, F',
                  'A, B',
                  'A, E, F',
                  'A, B, D, E, F',
                  'A, F',
                  'B, D, E',
                  'B, D, E, F',
                  'B, C, D, E',
                  'C, D, E']
       }
df_items_6 = pd.DataFrame(dic_6)
df_items_6.set_index('TID', inplace=True)
df_items_6

,Items
TID,
1,"A, B, C"
2,"A, C, D, E"
3,"A, B, D"
4,"A, C, F"
5,"A, B"
6,"A, E, F"
7,"A, B, D, E, F"
8,"A, F"
9,"B, D, E"


Usar Apriori y FP-Growth para encontrar los conjuntos de ítems frecuentes con mínimo soporte de 2. Tratar con varios valores de confianza. Ordenar los conjuntos resultados y analizar resultados de cada algoritmo. Comparar eficiencia de los dos procesos de mineria. Repetir con soporte de 3 y comparar los resultados.

In [32]:
list_transactions_6, items_6 = get_list_transactions(df_items_6['Items'])
list_transactions_6

[['A', 'B', 'C'],
 ['A', 'C', 'D', 'E'],
 ['A', 'B', 'D'],
 ['A', 'C', 'F'],
 ['A', 'B'],
 ['A', 'E', 'F'],
 ['A', 'B', 'D', 'E', 'F'],
 ['A', 'F'],
 ['B', 'D', 'E'],
 ['B', 'D', 'E', 'F'],
 ['B', 'C', 'D', 'E'],
 ['C', 'D', 'E']]

In [33]:
N = len(list_transactions_6)

In [34]:
min_soporte = 2
min_confianza = 0.6

In [35]:
itemsets, rules = apriori(list_transactions_6, min_support=min_soporte/N, min_confidence=min_confianza)
print(len(rules))
for r in rules:
  print(r)

32
{C} -> {A} (conf: 0.600, supp: 0.250, lift: 0.900, conv: 0.833)
{C} -> {D} (conf: 0.600, supp: 0.250, lift: 1.029, conv: 1.042)
{C} -> {E} (conf: 0.600, supp: 0.250, lift: 1.029, conv: 1.042)
{E} -> {D} (conf: 0.857, supp: 0.500, lift: 1.469, conv: 2.917)
{D} -> {E} (conf: 0.857, supp: 0.500, lift: 1.469, conv: 2.917)
{D} -> {B} (conf: 0.714, supp: 0.417, lift: 1.224, conv: 1.458)
{B} -> {D} (conf: 0.714, supp: 0.417, lift: 1.224, conv: 1.458)
{F} -> {A} (conf: 0.800, supp: 0.333, lift: 1.200, conv: 1.667)
{F} -> {E} (conf: 0.600, supp: 0.250, lift: 1.029, conv: 1.042)
{A, E} -> {D} (conf: 0.667, supp: 0.167, lift: 1.143, conv: 1.250)
{A, D} -> {E} (conf: 0.667, supp: 0.167, lift: 1.143, conv: 1.250)
{C, E} -> {D} (conf: 1.000, supp: 0.250, lift: 1.714, conv: 416666666.667)
{C, D} -> {E} (conf: 1.000, supp: 0.250, lift: 1.714, conv: 416666666.667)
{C} -> {D, E} (conf: 0.600, supp: 0.250, lift: 1.200, conv: 1.250)
{A, D} -> {B} (conf: 0.667, supp: 0.167, lift: 1.143, conv: 1.250)
{E,

In [36]:
patterns = pyfpgrowth.find_frequent_patterns(list_transactions_6, min_soporte/N)

In [37]:
rules = pyfpgrowth.generate_association_rules(patterns, min_confianza)
print(len(rules))
rules

26


{('A', 'B', 'D', 'E'): (('F',), 1.0),
 ('A', 'B', 'D', 'F'): (('E',), 1.0),
 ('A', 'B', 'E'): (('D',), 1.0),
 ('A', 'B', 'E', 'F'): (('D',), 1.0),
 ('A', 'B', 'F'): (('D', 'E'), 1.0),
 ('A', 'C', 'D'): (('E',), 1.0),
 ('A', 'C', 'E'): (('D',), 1.0),
 ('A', 'D'): (('E',), 0.6666666666666666),
 ('A', 'D', 'E', 'F'): (('B',), 1.0),
 ('A', 'D', 'F'): (('B', 'E'), 1.0),
 ('A', 'E'): (('D',), 0.6666666666666666),
 ('B',): (('D',), 0.7142857142857143),
 ('B', 'C', 'D'): (('E',), 1.0),
 ('B', 'C', 'E'): (('D',), 1.0),
 ('B', 'D'): (('E',), 0.8),
 ('B', 'D', 'F'): (('E',), 1.0),
 ('B', 'E'): (('D',), 1.0),
 ('B', 'E', 'F'): (('D',), 1.0),
 ('B', 'F'): (('D', 'E'), 1.0),
 ('C', 'D'): (('E',), 1.0),
 ('C', 'E'): (('D',), 1.0),
 ('C', 'F'): (('A',), 1.0),
 ('D', 'E'): (('B',), 0.6666666666666666),
 ('D', 'E', 'F'): (('B',), 1.0),
 ('D', 'F'): (('B', 'E'), 1.0),
 ('E', 'F'): (('A',), 0.6666666666666666)}

## 7. 

Usando el conjunto de datos del punto 5
- realizar la tabla de contingencia para las siguientes reglas: b→c , a→d , b→d , e→c , c→a

- usar las tablas de contigencia del punto anterior para computar y “realizar un ranking” de las reglas usando:
 - i. soporte
 - ii. confianza
 - iii. lift

In [38]:
def get_k2_metrics(s, minsop):
  list_transactions, items = get_list_transactions(s)
  df = get_support_df(items, list_transactions, get_all_2_itemsets)
  df = df[ (df['support'] > minsop) & (df['k'] == 2)]

  df_1 = get_support_df(items, list_transactions, get_all_1_itemsets)

  #desempaquetamos del array ['item'] -> 'item'
  df_1['k_itemsets'] = df_1['k_itemsets'].map(lambda s: s[0])

  return df.apply(add_a_b_metrics(df_1), axis=1)

In [39]:
def build_crosstab(rule, transactions=[]):
  a = rule[0]
  not_a = '~'+rule[0]

  b = rule[1]
  not_b = '~'+rule[1]

  f11 = 0 #'f11'
  f10 = 0 #'f10'
  f01 = 0 #'f01'
  f00 = 0 #'f00'

  N = len(transactions) #'N'
  set_rule = set(rule)

  for t in transactions:

    not_a_in_t =  not a in t
    not_b_in_t =  not b in t
    f11 = f11 + int(set_rule.issubset(t))
    f01 = f01 + int(b in t and not_a_in_t)
    f10 = f10 + int(a in t and not_b_in_t)
    f00 = f00 + int(not_a_in_t and not_b_in_t)


  fp1 = f11 + f01 #'f+1' 
  fp0 = f10 + f00 #'f+0'
  f1p = f11 + f10 #'f1+'
  f0p = f01 + f00 #'f0+'


  crosstab = { 
                b:      [f11, f01, fp1],
                not_b:  [f10, f00, fp0],
                '':     [f1p, f0p, N] 
             }

  metrics = {
      'soporte': f11/N,
      'confianza': f11/f1p,
      'lift': f11/(fp1*f1p)
  }

  return pd.DataFrame(crosstab, index= [a, not_a, '']), metrics

In [40]:
list_transactions_7, items_7 = get_list_transactions(df_items_5['Items'])

In [41]:
reglas = []

In [42]:
#b→c
ct, m = build_crosstab(['b', 'c'], list_transactions_7)
m['regla'] = 'b→c'
reglas.append(m)
print(m)
ct

{'soporte': 0.3, 'confianza': 0.42857142857142855, 'lift': 0.08571428571428572, 'regla': 'b→c'}


,c,~c,
b,3,4,7
~b,2,1,3
,5,5,10


In [43]:
#a→d
ct, m = build_crosstab(['a', 'd'], list_transactions_7)
m['regla'] = 'a→d'
reglas.append(m)
print(m)
ct

{'soporte': 0.4, 'confianza': 0.8, 'lift': 0.08888888888888889, 'regla': 'a→d'}


,d,~d,
a,4,1,5
~a,5,0,5
,9,1,10


In [44]:
#e→c
ct, m = build_crosstab(['e', 'c'], list_transactions_7)
m['regla'] = 'e→c'
reglas.append(m)
print(m)
ct

{'soporte': 0.2, 'confianza': 0.3333333333333333, 'lift': 0.06666666666666667, 'regla': 'e→c'}


,c,~c,
e,2,4,6
~e,3,1,4
,5,5,10


In [45]:
#c→a
ct, m = build_crosstab(['c', 'a'], list_transactions_7)
m['regla'] = 'c→a'
reglas.append(m)
print(m)
ct

{'soporte': 0.2, 'confianza': 0.4, 'lift': 0.08, 'regla': 'c→a'}


,a,~a,
c,2,3,5
~c,3,2,5
,5,5,10


In [46]:
pd.DataFrame(reglas)

,soporte,confianza,lift,regla
0,0.3,0.428571,0.085714,b→c
1,0.4,0.800000,0.088889,a→d
2,0.2,0.333333,0.066667,e→c
3,0.2,0.400000,0.080000,c→a


In [47]:
get_k2_metrics(df_items_5['Items'], minsop=0.25)


{'d': 9.0, 'c': 5.0, 'a': 5.0, 'e': 6.0, 'b': 7.0}


,k_itemsets,k,support_count,support,c(a->b),c(b->a),"I(a,b)"
0,"[d, c]",2.0,3.0,0.3,0.333333,0.600000,0.066667
1,"[d, a]",2.0,4.0,0.4,0.444444,0.800000,0.088889
2,"[d, e]",2.0,6.0,0.6,0.666667,1.000000,0.111111
3,"[d, b]",2.0,5.0,0.5,0.555556,0.714286,0.079365
6,"[c, b]",2.0,3.0,0.3,0.600000,0.428571,0.085714
7,"[a, e]",2.0,4.0,0.4,0.800000,0.666667,0.133333
8,"[a, b]",2.0,3.0,0.3,0.600000,0.428571,0.085714
9,"[e, b]",2.0,4.0,0.4,0.666667,0.571429,0.095238


## 8. 

Importar el conjunto de datos marketBasquet.csv. Extraer reglas de asociación utilizando diferentes valores mínimos de confíanza. Analice los resultados. Reporte los itemsets frecuentes con sus respectivos valores de soporte. Revise los valores de soporte reportados por el operador de itemsets frecuentes. Verifíquelos manualmente. ¿Qué problema se presenta? Extraiga reglas de asociación utilizando diferentes valores mínimos de confíanza. Analice los resultados. Reporte los itemsets frecuentes con sus respectivos valores de soporte. Analice los resultados y discuta las diferencias con el esquema que no posee un filtro de atributos.

In [48]:
uploaded = files.upload()

Saving marketBasket.csv to marketBasket.csv


In [49]:
!cat marketBasket.csv


ID;Leche;Cerveza;Pa�ales;Pan;Mantequilla;Galletas
1;true;true;true;false;false;false
2;true;false;false;true;true;false
3;true;false;true;false;false;true
4;false;false;false;true;true;true
5;false;true;true;false;false;true
6;true;false;true;true;true;false
7;false;false;true;true;true;false
8;false;true;true;false;false;false
9;true;false;true;true;true;false
10;false;true;false;false;false;true


In [50]:
#Eliminamos las primeras dos lineas (linea en blanco, la segunda linea tiene un error)
!sed '2d' marketBasket.csv > marketBasket1.csv

In [51]:
!cat marketBasket1.csv


1;true;true;true;false;false;false
2;true;false;false;true;true;false
3;true;false;true;false;false;true
4;false;false;false;true;true;true
5;false;true;true;false;false;true
6;true;false;true;true;true;false
7;false;false;true;true;true;false
8;false;true;true;false;false;false
9;true;false;true;true;true;false
10;false;true;false;false;false;true


In [52]:

df_market_basket = pd.read_csv('marketBasket1.csv', sep=';', names=['ID', 'Leche', 'Cerveza', 'Pañales','Pan', 'Mantequilla', 'Galletas'], index_col='ID')
df_market_basket

,Leche,Cerveza,Pañales,Pan,Mantequilla,Galletas
ID,,,,,,
1,True,True,True,False,False,False
2,True,False,False,True,True,False
3,True,False,True,False,False,True
4,False,False,False,True,True,True
5,False,True,True,False,False,True
6,True,False,True,True,True,False
7,False,False,True,True,True,False
8,False,True,True,False,False,False
9,True,False,True,True,True,False


In [53]:
transactions = df_market_basket.apply(lambda s: s.index[s].tolist(), axis=1).values.tolist()
transactions

[['Leche', 'Cerveza', 'Pañales'],
 ['Leche', 'Pan', 'Mantequilla'],
 ['Leche', 'Pañales', 'Galletas'],
 ['Pan', 'Mantequilla', 'Galletas'],
 ['Cerveza', 'Pañales', 'Galletas'],
 ['Leche', 'Pañales', 'Pan', 'Mantequilla'],
 ['Pañales', 'Pan', 'Mantequilla'],
 ['Cerveza', 'Pañales'],
 ['Leche', 'Pañales', 'Pan', 'Mantequilla'],
 ['Cerveza', 'Galletas']]

In [54]:
patterns = pyfpgrowth.find_frequent_patterns(transactions, 2)

In [55]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.6)
rules

{('Cerveza',): (('Pañales',), 0.75),
 ('Leche',): (('Mantequilla', 'Pan'), 0.6),
 ('Leche', 'Mantequilla'): (('Pan', 'Pañales'), 0.6666666666666666),
 ('Leche', 'Mantequilla', 'Pan'): (('Pañales',), 0.6666666666666666),
 ('Leche', 'Mantequilla', 'Pañales'): (('Pan',), 1.0),
 ('Leche', 'Pan'): (('Mantequilla', 'Pañales'), 0.6666666666666666),
 ('Leche', 'Pan', 'Pañales'): (('Mantequilla',), 1.0),
 ('Mantequilla', 'Pan'): (('Pañales',), 0.6),
 ('Mantequilla', 'Pan', 'Pañales'): (('Leche',), 0.6666666666666666),
 ('Mantequilla', 'Pañales'): (('Leche', 'Pan'), 0.6666666666666666),
 ('Pan', 'Pañales'): (('Leche', 'Mantequilla'), 0.6666666666666666)}

## 9. 

Importe el conjunto de datos credit-german.csv ([repositorio de machinelearning](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data))). Discretice los atributos numéricos en máximo 5 bins de igual tamaño. Aplique el algoritmo de reglas de asociación a este conjunto. Interprete las reglas producidas. Varíe los valores de soporte y de confianza ¿Qué sucede? Interprete las reglas producidas y escoja las que en su concepto son las más interesantes para el problema (justifíque).

In [56]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric'

--2021-04-30 02:09:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102000 (100K) [application/x-httpd-php]
Saving to: ‘german.data-numeric’

german.data-numeric 100%[===================>]  99.61K   517KB/s    in 0.2s    

2021-04-30 02:09:10 (517 KB/s) - ‘german.data-numeric’ saved [102000/102000]



In [57]:
!cat german.data-numeric

   1   6   4  12   5   5   3   4   1  67   3   2   1   2   1   0   0   1   0   0   1   0   0   1   1 
   2  48   2  60   1   3   2   2   1  22   3   1   1   1   1   0   0   1   0   0   1   0   0   1   2 
   4  12   4  21   1   4   3   3   1  49   3   1   2   1   1   0   0   1   0   0   1   0   1   0   1 
   1  42   2  79   1   4   3   4   2  45   3   1   2   1   1   0   0   0   0   0   0   0   0   1   1 
   1  24   3  49   1   3   3   4   4  53   3   2   2   1   1   1   0   1   0   0   0   0   0   1   2 
   4  36   2  91   5   3   3   4   4  35   3   1   2   2   1   0   0   1   0   0   0   0   1   0   1 
   4  24   2  28   3   5   3   4   2  53   3   1   1   1   1   0   0   1   0   0   1   0   0   1   1 
   2  36   2  69   1   3   3   2   3  35   3   1   1   2   1   0   1   1   0   1   0   0   0   0   1 
   4  12   2  31   4   4   1   4   1  61   3   1   1   1   1   0   0   1   0   0   1   0   1   0   1 
   2  30   4  52   1   1   4   2   3  28   3   2   1   1   1   1   0   1   0   0  

In [58]:

credit = np.genfromtxt('german.data-numeric')
#X,y = credit[:,:-1], credit[:,-1:].squeeze()
credit

array([[ 1.,  6.,  4., ...,  0.,  1.,  1.],
       [ 2., 48.,  2., ...,  0.,  1.,  2.],
       [ 4., 12.,  4., ...,  1.,  0.,  1.],
       ...,
       [ 4., 12.,  2., ...,  0.,  1.,  1.],
       [ 1., 45.,  2., ...,  0.,  1.,  2.],
       [ 2., 45.,  4., ...,  0.,  1.,  1.]])

In [59]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html
#  Discretice los atributos numéricos en máximo 5 bins de igual tamaño
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='onehot', strategy='uniform')


In [60]:
# https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer
# las columnas 1, 3, 9 serán discretizadas en maximo 5 bins
all_cols =  [i for i in range(0, 25, 1)] 
cols_to_discretize = [1, 3, 9]
cols_to_ignore = list(filter(lambda i: not i in cols_to_discretize, all_cols))

'''
ct = ColumnTransformer([('nbin', discretizer, cols_to_discretize),
                        ('passthrough',  'passthrough', cols_to_ignore) ])
'''

ct = ColumnTransformer([('nbin', discretizer, all_cols)])

In [61]:
credit_disc = ct.fit_transform(credit)
credit_disc

<1000x125 sparse matrix of type '<class 'numpy.float64'>'
	with 25000 stored elements in Compressed Sparse Row format>

In [62]:
headers = []
for a in range(1,26):
  for i in range(5):
    headers.append(f'A{a}_{i}')

In [63]:
df_credit_disc = pd.DataFrame.sparse.from_spmatrix(credit_disc, columns=headers, index=range(1000))
df_credit_disc

,A1_0,A1_1,A1_2,A1_3,A1_4,A2_0,A2_1,A2_2,A2_3,A2_4,A3_0,A3_1,A3_2,A3_3,A3_4,A4_0,A4_1,A4_2,A4_3,A4_4,A5_0,A5_1,A5_2,A5_3,A5_4,A6_0,A6_1,A6_2,A6_3,A6_4,A7_0,A7_1,A7_2,A7_3,A7_4,A8_0,A8_1,A8_2,A8_3,A8_4,...,A18_0,A18_1,A18_2,A18_3,A18_4,A19_0,A19_1,A19_2,A19_3,A19_4,A20_0,A20_1,A20_2,A20_3,A20_4,A21_0,A21_1,A21_2,A21_3,A21_4,A22_0,A22_1,A22_2,A22_3,A22_4,A23_0,A23_1,A23_2,A23_3,A23_4,A24_0,A24_1,A24_2,A24_3,A24_4,A25_0,A25_1,A25_2,A25_3,A25_4
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
996,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [64]:
df_credit_disc_1 = df_credit_disc.iloc[:10, :10]
df_credit_disc_1

,A1_0,A1_1,A1_2,A1_3,A1_4,A2_0,A2_1,A2_2,A2_3,A2_4
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [65]:
transactions_test = df_credit_disc_1.apply(lambda s: s.index[s.values == 1].tolist(), axis=1).values.tolist()
transactions_test

[['A1_0', 'A2_0'],
 ['A1_1', 'A2_3'],
 ['A1_4', 'A2_0'],
 ['A1_0', 'A2_2'],
 ['A1_0', 'A2_1'],
 ['A1_4', 'A2_2'],
 ['A1_4', 'A2_1'],
 ['A1_1', 'A2_2'],
 ['A1_4', 'A2_0'],
 ['A1_1', 'A2_1']]

In [66]:
transactions_9 = df_credit_disc.apply(lambda s: s.index[s.values == 1].tolist(), axis=1).values.tolist()

In [67]:
min_soporte_9 = 1000*0.6
min_confianza = 0.8

In [68]:
patterns_9 = pyfpgrowth.find_frequent_patterns(transactions_9, min_soporte_9)
rules_9 = pyfpgrowth.generate_association_rules(patterns_9, min_confianza)
rules_9

{('A11_4', 'A13_0', 'A15_0'): (('A22_0',), 0.9778434268833087),
 ('A11_4', 'A13_0', 'A15_0', 'A18_4'): (('A19_0', 'A22_0'),
  0.9791332263242376),
 ('A11_4', 'A13_0', 'A15_0', 'A18_4', 'A19_0'): (('A22_0',),
  0.9791332263242376),
 ('A11_4', 'A13_0', 'A15_0', 'A18_4', 'A22_0'): (('A19_0',), 1.0),
 ('A11_4', 'A13_0', 'A15_0', 'A19_0'): (('A22_0',), 0.9800307219662059),
 ('A11_4', 'A13_0', 'A15_0', 'A19_0', 'A22_0'): (('A18_4',),
  0.9561128526645768),
 ('A11_4', 'A13_0', 'A15_0', 'A22_0'): (('A19_0',), 0.9637462235649547),
 ('A11_4', 'A13_0', 'A18_4', 'A19_0'): (('A22_0',), 0.9780564263322884),
 ('A11_4', 'A13_0', 'A18_4', 'A19_0', 'A22_0'): (('A15_0',),
  0.9775641025641025),
 ('A11_4', 'A13_0', 'A18_4', 'A22_0'): (('A19_0',), 1.0),
 ('A11_4', 'A13_0', 'A19_0', 'A22_0'): (('A15_0',), 0.9755351681957186),
 ('A11_4', 'A13_0', 'A22_0'): (('A15_0',), 0.9706744868035191),
 ('A11_4', 'A15_0'): (('A22_0',), 0.9782608695652174),
 ('A11_4', 'A15_0', 'A17_0'): (('A22_0',), 0.9770773638968482),
 

**Justificación**